## Übung Kognitive Robotik - Anwendungsfall Robotergreifen

### Einleitung

Eine Einführung in den behandelten Anwendungsfall und die verwendeten Daten wird im Kurselement "Anwendungsfall Robotergreifen" gegeben. Sie können gerne noch einmal zu diesem Kurselement zurückkehren, bevor Sie mit der praktischen Übung beginnen.   

### Schritt 1: Erzeugung synthetischer Daten

Wie bereits im Einführungsmodul erläutert wurde, besteht eine große Herausforderung beim Einsatz von 
Machine Learning darin, dass eine ausreichende Datengrundlage für das Training der ML-Modelle vorhanden sein muss. 
Da das Aufnehmen und Labeln von realen Daten für viele Anwendungsfälle mit beträchtlichem Aufwand verbunden ist, kommen für das Training von ML-Modellen vermehrt synthetische Daten zum Einsatz. Dies sind Daten, die mithilfe von Simulationen oder speziellen Algorithmen künstlich generiert wurden.

#### Aufgabe 1.1: 

Für den ersten Teil der Übung soll nun ein recht simpler synthetischer Datensatz erzeugt werden. Zu diesem Zweck sollen synthetische Bilder generiert werden, auf denen jeweils ein einfaches geometrisches Objekt zu sehen ist. Im weiteren Verlauf der Übung soll dann eine Objekterkennung für diesen Datensatz umgesetzt werden. Führen Sie die folgenden Codezellen aus. Welche Informationen über den Datensatz können Sie aus der zweiten Codezelle ziehen? 

In [ ]:
import numpy as np
import os
import json
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import itertools

import tensorflow as tf
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Activation, Dropout, Convolution2D, MaxPooling2D, Flatten, InputLayer, MaxPool2D, Input
from keras.losses import MeanSquaredError, CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from skimage import transform
from skimage.transform import rotate
from skimage.util import random_noise
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Parameter Synthetische Daten

num_imgs = 1000

img_size = 224
min_object_size = 40
max_object_size = 80

num_classes = 3
class_labels = ['Rechteck', 'Kreis', 'Dreieck']

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%load Loesung/Loesung_1_1.py

#### Aufgabe 1.2:

Neben den Bilddaten müssen dafür auch die entsprechenden Labels erzeugt werden. Wie im Kurselement "Objekterkennung mit tiefen neuronalen Netzen" erläutert wurde, ist dies hier für jedes Bild zum einen die Position des Objekts in Form einer Bounding Box und zum anderen die Klasse des Objekts. Betrachten Sie den folgenden Code und denken Sie zurück an das Kurselement "Objekterkennung mit tiefen neuronalen Netzen". In welcher Form werden die Bilder und die Labels hier gespeichert?  

In [ ]:
# Array für die erzeugten Bilder
imgs = np.zeros((num_imgs, img_size, img_size, 3), dtype=np.uint8)
print("imgs:", imgs.shape)

# Array für die zugehörigen Bounding Boxen
bboxes = np.zeros((num_imgs, 4))
print("bboxes:", bboxes.shape)

# Array für die zugehörigen Klassen
classes = np.zeros((num_imgs, num_classes), dtype=int)
print("classes:", classes.shape)

min_distance_wall = 5 # minimaler Abstand der Objekte vom Bildrand

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_1_2.py

#### Aufgabe 1.3: 

Betrachten Sie den folgenden Code für die Erzeugung der synthetischen Bilddaten. Wie wird hier ein synthetisches Bild generiert? 

In [ ]:
# Array für die erzeugten Bilder
imgs = np.zeros((num_imgs, img_size, img_size, 3), dtype=np.uint8)
print("imgs:", imgs.shape)

# Array für die zugehörigen Bounding Boxen
bboxes = np.zeros((num_imgs, 4))
print("bboxes:", bboxes.shape)

# Array für die zugehörigen Klassen
classes = np.zeros((num_imgs, num_classes), dtype=int)
print("classes:", classes.shape)

min_distance_wall = 5 # minimaler Abstand der Objekte vom Bildrand

for i_img in range(num_imgs):
    
    img = np.ones((img_size,img_size,3), np.uint8) * 255 # Erzeugung eines weißen Bildes

    obj_class = np.random.randint(num_classes) #zufällige Wahl der Objektklasse
    
    c1, c2, c3 = np.random.randint(0, 150, size=3)
    mycolor =  (int(c1), int(c2),int(c3)) # zufällige Farbe 
    
    classes[i_img, obj_class] = 1 # Speichern der Klasse, One-Hot-Encoding
    
    if obj_class == 0: #Rechteck
        w, h = np.random.randint(min_object_size, max_object_size, size=2) # Breite und Höhe
        x = np.random.randint(min_distance_wall, img_size - w -min_distance_wall) # x-Position im Bild
        y = np.random.randint(min_distance_wall, img_size - h -min_distance_wall) # y-Position im Bild
        cv.rectangle(img, (x+0,y+0), (x+w-1,y+h-1), mycolor, -1) # Erzeugung Rechteck im Bild
        bbox = [x, y+h, x+w, y] # Bounding Box 
                        
    elif obj_class == 1: # Kreis   
        r = 0.5 * np.random.randint(min_object_size, max_object_size) # Radius
        x = np.random.randint(r+2, img_size-r-2) # x-Position im Bild
        y = np.random.randint(r+2, img_size-r-2) # y-Position im Bild
        cv.circle(img,(x,y), int(r), mycolor, -1) # Erzeugung Kreis im Bild
        bbox = [x-r, y+r, x+r, y-r] # Bounding Box
        
    elif obj_class == 2: # Dreieck
        w, h = np.random.randint(min_object_size, max_object_size, size=2) # Breite und Höhe
        x = np.random.randint(min_distance_wall, img_size - w -min_distance_wall) # x-Position im Bild
        y = np.random.randint(min_distance_wall, img_size - h -min_distance_wall) # y-Position im Bild
        triangle_cnt = np.array( [(x,y), (x+w,y), (x,y+h)] )
        cv.drawContours(img, [triangle_cnt], 0, mycolor, -1) # Erzeugung Dreieck im Bild
        bbox = [x, y+h, x+w, y] # Bounding Box
        
    imgs[i_img] = img
    bboxes[i_img] = bbox
 

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_1_3.py

### Schritt 2: Exploration der Daten 

#### Aufgabe 2.1: 

Überprüfen Sie die erzeugten synthetischen Daten stichprobenhaft. Enthält jedes Bild eines der drei geometrischen Objekte? Sind die Labels, d. h. die Bounding Boxen und die Klassen korrekt? Sind alle Objekte vollständig im Bild? Etc. 

In [ ]:
temp_imgs = np.copy(imgs) # Erzeugung einer Kopie der Bilder 

# Plotten von einigen Beispielbildern
def plot_example_imgs(imgs,bboxes,classes):
    
    plt.figure(figsize=(16, 16))
    for i_subplot in range(1, 17):
        plt.subplot(4, 4, i_subplot)
        i = np.random.randint(len(imgs))
        
        # Bild 
        plt.imshow(imgs[i], origin='lower')
        ax = plt.gca()
        
        # Bounding Box
        x, y = bboxes[i,0], bboxes[i,3]
        w, h = (bboxes[i,2] - bboxes[i,0]), (bboxes[i,1] - bboxes[i,3])
        box = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor='g', facecolor="none")
        ax.add_patch(box)
        
        # Klasse
        class_num = np.argmax(classes[i])
        plt.text(imgs.shape[2]-100,imgs.shape[1]+5, class_labels[class_num], color='g')
        
plot_example_imgs(temp_imgs,bboxes,classes)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_2_1.py

#### Aufgabe 2.2:

Wie sind die Daten auf die Klassen verteilt?

In [ ]:
class_dist = np.sum(classes, axis=0)/np.sum(classes)
for i in range(3):
 print(class_labels[i], class_dist[i]*100, "%")

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_2_2.py

### Schritt 3: Datenvorverarbeitung

#### Aufgabe 3.1: 

Im ersten Schritt der Datenvorverarbeitung werden die Pixelwerte neu skaliert. Was genau wird hier gemacht und warum? 

In [ ]:
def pixel_normalization(imgs):
   
    h = imgs.shape[1]
    b = imgs.shape[2]
    
    imgs = tf.cast(imgs, tf.float32)
    s = tf.constant(255.0,  dtype = tf.float32)
    imgs_scaled = tf.math.divide(imgs,s)
    
    print("Vorher:")
    print(tf.reshape(imgs[0],(h,b,3)))
    print("Nachher:")
    print(tf.reshape(imgs_scaled[0],(h,b,3)))
    
    return imgs_scaled

imgs_scaled = pixel_normalization(imgs)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_3_1.py

#### Aufgabe 3.2

Im nächsten Schritt der Datenvorverarbeitung werden auch die Bounding Boxen neu skaliert. Was genau wird hier gemacht und warum?  

In [ ]:
def bbox_normalization(bboxes, imgs):
    
    print("Vorher: min", np.min(bboxes), "max", np.max(bboxes))
    
    h = imgs.shape[1]
    b = imgs.shape[2]
    
    bboxes[:,0] = bboxes[:,0]/b
    bboxes[:,1] = bboxes[:,1]/h
    bboxes[:,2] = bboxes[:,2]/b
    bboxes[:,3] = bboxes[:,3]/h
    
    print("Nachher: min", np.min(bboxes), "max", np.max(bboxes))
    
    return bboxes

bboxes = bbox_normalization(bboxes, imgs)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_3_2.py

### Schritt 4: Sampling

#### Aufgabe 4.1:

Als Nächstes muss der erzeugte synthetische Datensatz in Trainingsdaten und Testdaten aufgeteilt werden. Dabei sollen 80 % der Daten als Trainingsdaten und 20 % der Daten als Testdaten verwendet werden. Ergänzen Sie den gegebenen Code.

In [ ]:
# Aufteilung der Daten in Trainings- und Testdaten 

n_train = int(0.8 * num_imgs) # Anzahl der Bilder, die zum Trainingsdatensatz gehören sollen

# Trainingsdaten
x_train = imgs_scaled[:n_train] # Die ersten n_train Bilder gehören zum Trainingsdatensatz
bboxes_train = bboxes[:n_train]
class_train = classes[:n_train]

#Testdaten
x_test =      # Die restlichen Bilder gehören zum Testdatensatz
bboxes_test = 
class_test = 

print("Anzahl Bilder Trainingsdatensatz: ", len(x_train))
print("Anzahl Bilder Testdatensatz: ", len(x_test))

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%load Loesung/Loesung_4_1.py

### Schritt 5: Umsetzung Objekterkennung mit einem Convolutional Neural Network

Im Folgenden soll eine Objekterkennung mit einem Convolutional Neural Network (CNN) für die erzeugten synthetischen Daten umgesetzt werden. Auf die Funktionsweise von CNNs wird später im Modul Qualitätsprüfung eingegangen. Hier werden Sie zunächst als Black Box betrachtet. Der Aufbau des verwendeten Netzes wurde im Kurselement "Objekterkennung mit tiefen neuronalen Netzen" genauer erläutert und ist zur Erinnerung in der folgenden Grafik noch einmal dargestellt.  

<img src="img/Netzarchitektur.png" width=500 />


#### Aufgabe 5.1: 

Betrachten Sie die Abbildung der Netzwerkarchitektur und versuchen Sie den gegebenen Code für die Erstellung des Modells nachzuvollziehen. 

Wie Sie dem Code entnehmen können, werden drei verschiedene Aktivierungsfunktionen für die verschiedenen Schichten des Netzes verwendet. Welche sind das und wie sehen diese Aktivierungsfunktionen aus? Warum macht es Sinn diese für die entsprechenden Schichten des Netzes zu verwenden? 

Informationen zu verschiedenen Aktivierungsfunktionen und Hinweise zur Lösung der Aufgabe finden Sie u. a. hier: https://www.geeksforgeeks.org/activation-functions-neural-networks/  


In [ ]:
def build_model():

    # Eingabe
    inputs = Input(shape=x_train.shape[1:])
    
    # CNN
    x = Convolution2D(filters=32,kernel_size=(3,3), activation="relu")(inputs)
    x = MaxPool2D(pool_size=(3,3))(x)
    x = Convolution2D(filters=16,kernel_size=(3,3), activation="relu")(x)
    x = MaxPool2D(pool_size=(3,3))(x)
    x = Convolution2D(filters=8,kernel_size=(3,3), activation="relu")(x)
    x = MaxPool2D(pool_size=(3,3)) (x)
    x = Flatten() (x)

    # Lokalisierung (Regressionsproblem)
    x_1 = Dense(128, activation="relu")(x)
    x_1 = Dropout(0.3)(x_1)
    x_1 = Dense(64,  activation="relu")(x_1)
    x_1 = Dropout(0.3)(x_1)
    output_bb = Dense(4, activation="sigmoid", name='output_bb')(x_1) # Ausgabe Bounding Box

    # Klassifizierung (Klassifikationsproblem)
    x_2 = Dense(64, activation="relu")(x)
    x_2 = Dropout(0.3)(x_2)
    x_2 = Dense(32, activation="relu")(x_2)
    x_2 = Dropout(0.3)(x_2)
    output_class = Dense(3, activation="softmax", name='output_class')(x_2) # Ausgabe Wahrscheinlichkeit Objektklasse

    model = Model(inputs=inputs, outputs=[output_bb, output_class])
    return model

model = build_model()

# Speichern der zufällig initialisierten Gewichte des Modells 
model.save_weights('model_default_weights.h5')

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_5_1.py

#### Aufgabe 5.2:

Betrachten Sie den folgenden Code und führen Sie ihn aus, um das erstellte Modell zu trainieren. Das Training des Modells benötigt ca. 10 Minuten. Nutzen Sie diese Zeit, um die folgenden Fragen zu beantworten.

a) Wie wird hier der Fehler zwischen den Vorhersagen des Netzes und den gegebenen Labels (Loss) berechnet? 

b) Welcher Anteil der Trainingsdaten wird für die Validierung verwendet? 

c) Welche Gewichte werden während des Trainings des Modells gespeichert? 

Falls das Training zu lange dauert, können Sie auch die Musterlösung laden.     

In [ ]:
lr = 0.001 # Lernrate
num_epochs = 40 # Anzahl Epochen
batch_size = 64 # Batch size

# Gewichtung der Fehler 
w_1 = 10.0 # Lokalisierung
w_2 = 1.0 # Klassifizierung

# Laden der zufälligen Startgewichte des Netzes  
model.load_weights('model_default_weights.h5')

opt = Adam(learning_rate=lr)

model.compile(loss={'output_bb' : 'mean_squared_error', 'output_class' : 'categorical_crossentropy'}, 
              loss_weights=[w_1,w_2],
              optimizer=opt, 
              metrics=['accuracy'])

model_checkpoint_callback = ModelCheckpoint(
                            filepath="my_model.h5",
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            save_weights_only=False)

history = model.fit(
                    x_train,
                    (bboxes_train, class_train),
                    batch_size=batch_size,
                    epochs=num_epochs,
                    callbacks=[model_checkpoint_callback],
                    validation_split=0.2)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe a) laden. 

In [ ]:
%run Loesung/Loesung_5_2_a.py

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe b) laden. 

In [ ]:
%run Loesung/Loesung_5_2_b.py

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe c) laden. 

In [ ]:
%run Loesung/Loesung_5_2_c.py

In [ ]:
# Sie können entweder mit dem von Ihnen trainierten Netz weiterarbeiten oder mit der Musterlösung 

# Von Ihnen trainiertes Netz laden 
model.load_weights('my_model.h5')

# Musterlösung laden 
#model.load_weights('solution_model.h5')

### Schritt 6: Evaluation Objekterkennung

#### Aufgabe 6.1: 

Testen Sie das trainierte neuronale Netz auf einigen Beispielbildern aus dem Testdatensatz. 

a) Ergänzen Sie dazu den folgenden Code. 

b) Wie gut funktioniert die Lokalisierung und die Klassifikation der Objekte?     

In [ ]:
# Wähle zufällig 16 Bilder aus dem Testdatensatz und plotte die korrekten und die vom Netz vorhergesagten Bboxes und Klassen
def plot_example_imgs_pred(x_test, bboxes_test, class_test, img_size, model):
    
    # Wähle zufällig 16 Bilder aus dem Testdatensatz
    myPlotSample = np.array(x_test)
    rnd_sample = np.random.randint(len(myPlotSample), size=16)
    images_show = myPlotSample[rnd_sample]
    
    # korrekte Bounding Boxen und Klassen
    true_bboxes = bboxes_test[rnd_sample]*img_size
    true_class = class_test[rnd_sample]
    
    # Wende das trainierte neuronale Netz an, um Bounding Boxen und Objektklassen für die Testbilder vorherzusagen   
    pred_y = model.predict(images_show)
    pred_bboxes = pred_y[0]*img_size
    pred_class = pred_y[1]
    
    # Plotte testbilder
    plt.figure(figsize=(16, 16))
    for i in range(16):
    
        plt.subplot(4, 4, i+1)
        plt.imshow(images_show[i], origin='lower')
        ax = plt.gca()
    
        # korrekte Bounding Boxen (grün)
        x, y = true_bboxes[i,0], true_bboxes[i,3]
        w, h = (true_bboxes[i,2] - true_bboxes[i,0]), (true_bboxes[i,1] - true_bboxes[i,3])
        box_true = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor='g', facecolor="none")     
        ax.add_patch(box_true)
        
        # vorhergesagte Bounding Boxen (rot) 
        # Ergänzen Sie den Code
    
        # vorhergesagte und korrekte Klasse
        predicted_class_num = np.argmax(pred_class[i])
        true_class_num = np.argmax(true_class[i])
        plt.text(0,x_test.shape[1]+5,class_labels[predicted_class_num], color='r') # vorhergesagte Klasse (grün) 
        plt.text(x_test.shape[2]-100,x_test.shape[1]+5, class_labels[true_class_num], color='g') # korrekte Klasse (rot)
    
    return

plot_example_imgs_pred(x_test, bboxes_test, class_test, img_size, model)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe a) laden. 

In [ ]:
%load Loesung/Loesung_6_1_a.py

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe b) laden. 

In [ ]:
%run Loesung/Loesung_6_1_b.py

#### Aufgabe 6.2:

In der vorherigen Aufgabe wurde die Performance des trainierten neuronalen Netzes qualitativ anhand einiger Beispielbilder beurteilt. In dieser und der folgenden Aufgabe soll die Performance nun anhand gängiger Metriken
quantitativ beurteilt werden. Zunächst soll dabei die Performance der Lokalisierung anhand der IoU (intersection over union) betrachtet werden. Für eine Erklärung von IoU sei auf das Kurselement "Methoden zur Evaluation der Objekterkennung" verwiesen.   

Standardmäßig wird eine Bounding Box als korrekt vorhergesagt betrachtet, wenn IoU >= 0.5 ist. Teilweise wird aber auch die Performance für verschiedene Schwellwerte angegeben. Für wie viel Prozent der Testdaten ist IoU >= 0.25, 0.5, 0.75, 0.95. Was sagt dies über die Performance des Netzes bei der Lokalisierung aus?  

In [ ]:
# Funktion für die Berechnung IoU
def calc_IoU(box_true, box_pred):
    # Fläche Überschneidung von vorhergesagter und korrekter Bounding Box
    x1 = max(box_true[0],box_pred[0])
    y1 = min(box_true[1],box_pred[1])
    x2 = min(box_true[2],box_pred[2])
    y2 = max(box_true[3],box_pred[3])
    intersection_area = max(0, x2 -  x1) * max(0, y1 - y2)
    # Fläche korrekte Bounding Box 
    true_box_area = (box_true[2] - box_true[0]) * (box_true[1] - box_true[3])
    # Fläche vorhergesagte Bounding Box
    pred_box_area = (box_pred[2] - box_pred[0]) * (box_pred[1] - box_pred[3])
    # Berechnung Intersection over Union
    IoU = intersection_area / (true_box_area + pred_box_area - intersection_area)
    return IoU

# Funktion für die Evaluation 
def eval_bbox_prediction(bbox_true, bbox_pred, threshold):
    iou_test = np.zeros(len(bbox_true))
    for j in range(len(bbox_true)):
        iou_test[j] = calc_IoU(bbox_true[j],bbox_pred[j])
    correct = np.sum(iou_test >= threshold)
    print("Korrekt vorhergesagte Bounding Boxen: ", correct, "von", len(bbox_true))

In [ ]:
# Evaluation der Lokalisierung auf den Testdaten 
threshold = 0.5 # Schwellwert IoU, ab dem Bounding Box als korrekt vorhergesagt betrachtet wird 

box_true = bboxes_test
box_pred = model.predict(x_test)[0]

eval_bbox_prediction(box_true,box_pred,threshold)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_6_2.py

#### Aufgabe 6.3:

Als Nächstes soll die Performance des Netzes bei der Klassifikation mithilfe einer Konfusionsmatrix quantitativ beurteilt werden. Bewerten Sie das Ergebnis.      

In [ ]:
def eval_class_prediction(true_class,pred_class):
    true_class_max = np.argmax(true_class,axis=1)
    pred_class_max = np.argmax(pred_class,axis=1)
   
    N_correct = np.sum(true_class_max == pred_class_max)
    print('Korrekt vorhergesagte Klassen: ', N_correct, 'von', len(x_test))
    
    cm = confusion_matrix(y_true=true_class_max, y_pred=pred_class_max)
    cmap = cmap=plt.cm.Reds
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.xticks([0,1,2],class_labels)
    plt.yticks([0,1,2],class_labels)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="black")

    plt.ylabel('Korrekte Klasse')
    plt.xlabel('Vorhergesagte Klasse')
    plt.title('Konfusionsmatrix')

In [ ]:
# Evaluation der Klassifikation
eval_class_prediction(class_test, model.predict(x_test)[1])

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_6_3.py

#### Aufgabe 6.4:

Als Nächstes soll getestet werden, wie sich die Performance des Netzes auf leicht veränderten Bildern verhält. Dazu wird künstliches Rauschen auf die Bilder gelegt. Experimentieren Sie mit der Menge an Rauschen und beobachten Sie, wie sich die Performance des Netzes verhält. 

In [ ]:
amount_of_noise = 0.01 # Menge an Rauschen

def noisy_image(input_imgs):
    noisy_imgs = np.copy(input_imgs)
    for i in range(len(input_imgs)):
        # Hinzufügen von "salt and pepper noise"
        noisy_imgs[i] = random_noise(noisy_imgs[i], mode='s&p', clip=True, amount=amount_of_noise)
    return noisy_imgs

x_test_noisy = noisy_image(x_test)

# Plotte Beispielbild mit Rauschen
plt.imshow(x_test_noisy[0], origin='lower')

In [ ]:
eval_bbox_prediction(bboxes_test,model.predict(x_test_noisy)[0],threshold=0.5)
eval_class_prediction(class_test, model.predict(x_test_noisy)[1])

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_6_4.py

### Schritt 7: Augmentierung der Daten

Die schlechte Extrapolationsfähigkeit von neuronalen Netzen wird insbesondere dann zum Problem, wenn der für das Training vorhandene Datensatz eher klein ist und hauptsächlich sehr ähnliche Daten enthält. In diesem Fall setzt man häufig sogenannte Augmentierung (engl. Augmentation) ein, um die Varianz und die Menge der Trainingsdaten zu erhöhen und so die Genauigkeit und die Robustheit des gelernten Machine Learning Modells zu verbessern. Bei der Augmentierung von Bilddaten (Image Augmentation) werden, basierend auf den vorhandenen Bildern, neue zusätzliche Bilder für das Training erzeugt. Dazu werden verschiedene Operationen wie zufällige Rotation, Spiegelung, Hinzufügen von künstlichem Rauschen, Veränderung des Kontrasts, etc. auf die ursprünglichen Bilder angewandt. 

Beim Einsatz von synthetischen Daten führt die schlechte Extrapolationsfähigkeit von neuronalen Netzen häufig dazu, dass ein Modell, welches auf synthetischen Daten trainiert wurde und auf diesen eine sehr gute Performance erreicht, auf realen Daten eine deutlich schlechtere Performance zeigt. Dies liegt daran, dass sich die synthetischen Daten eigentlich immer in gewisser Weise von den realen Daten unterscheiden. Auch hier kann Augmentierung helfen ein robusteres Machine Learning Modell zu trainieren, welches besser generalisiert und auch in der Realität gute Vorhersagen liefert.

#### Aufgabe 7.1:

Wie bereits erläutert, können Bilder auf unterschiedliche Weise augmentiert werden. Im Folgenden sind vier Funktionen für die Augmentierung der synthetischen Bilder definiert. Wie verändern diese Funktionen die ursprünglichen Bilder?   

In [ ]:
# Definition verschiedener Augmentierungsfunktionen  
    
def brightness(image):
    max_val = 1-np.min(image)-0.4
    bright = np.random.random(1)*max_val
    im_bright = image + bright
    im_bright = np.clip(im_bright, 0.0, 1.0)
    return im_bright

def darkness(image):
    max_val = np.min(image)
    dark = np.random.random(1)*max_val
    im_dark = image - dark
    im_dark = np.clip(im_dark, 0.0, 1.0)
    return im_dark

def noise(image):
    amount = np.random.uniform(0.0,0.05,1)
    im_noise = random_noise(image, mode='s&p', clip=True, amount=amount[0])
    return im_noise 
    
def blur(image):
    rand = np.random.randint(0,5)
    im_blur = cv.GaussianBlur(image,(5,5), rand)
    return(im_blur)

# Testbild augmentieren
original_img = x_test[0]
augmented_img = brightness(original_img) # brightness kann hier durch die anderen definierten Funktionen ersetzt werden

# Visualisierung 
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1, title="original")
plt.imshow(original_img, origin="lower")
plt.subplot(1, 2, 2, title="augmented")
plt.imshow(augmented_img, origin="lower")

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%load Loesung/Loesung_7_1.py

#### Aufgabe 7.2:

Im nächsten Schritt soll basierend auf dem Trainingsdatensatz ein gleich großer Datensatz mit augmentierten Bildern erzeugt werden. Betrachten Sie den folgenden Code und führen Sie ihn aus. Wie werden die augmentierten Bilder erzeugt?

In [ ]:
# Erzeugung eines Datensatzes mit augmentierten Bildern

def create_augmented_imgs(imgs,bboxes,classes):
    
    num_augimgs = len(imgs) # Anzahl zu erzeugender augmentierter Bilder

    # Platzhalter für augmentierte Bilder, Bounding Boxen und Klassen
    aug_imgs = np.ones(shape=imgs.shape)
    aug_bboxes = np.ones(shape=(num_augimgs,4))
    aug_classes = np.ones(shape=(num_augimgs,3))

    temp_train_imgs = np.array(imgs)

    for i in range(num_augimgs):
        sample = np.random.randint(len(temp_train_imgs)) # zufällige Wahl eines Bildes aus dem Trainingsdatensatz
        aug_img = temp_train_imgs[sample]
        aug = np.random.random(4) # zufällige Auswahl der Operationen, die auf das Bild angewandt werden
        if (aug[0]>0.5):
            aug_img = brightness(aug_img) # Aufhellung
        if (aug[1]>0.5):
            aug_img = darkness(aug_img) # Verdunklung
        if (aug[2]>0.5):
            aug_img = noise(aug_img) # Rauschen
        if (aug[3]>0.5):
            aug_img = blur(aug_img) # Unschärfe
        aug_imgs[i] = aug_img
        aug_bboxes[i] = bboxes[sample]
        aug_classes[i] = classes[sample]
    return aug_imgs, aug_bboxes, aug_classes

# Erzeugung augmentierter Bilddaten 
aug_imgs, aug_bboxes, aug_classes = create_augmented_imgs(x_train, bboxes_train, class_train)

# Plotten einiger Beispielbilder
N,h,b,c = aug_imgs.shape
size = np.array([b,h,b,h])
plot_example_imgs(aug_imgs, aug_bboxes*size, aug_classes)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_7_2.py

#### Aufgabe 7.3:

Nun soll ein neuer Trainingsdatensatz erstellt werden, der sowohl die ursprünglichen Bilder, wie auch die augmentierten Bilder enthält. Ergänzen Sie den folgenden Code.    

In [ ]:
# Erstellung eines neuen Trainingsdatensatzes mit den originalen und den augmentierten Bildern  

x_train_aug = np.concatenate((x_train,aug_imgs), axis=0)
bboxes_train_aug = 
class_train_aug = 

# Durchmischung der Bilder
x_train_aug, bboxes_train_aug, class_train_aug = shuffle(x_train_aug, bboxes_train_aug, class_train_aug)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%load Loesung/Loesung_7_3.py

### Schritt 8: Training des Modells mit dem neuen Datensatz

#### Aufgabe 8.1:

Im Folgenden ist der Code aus Aufgabe 5.2 (Training des Modells auf dem ursprünglichen Datensatz) gegeben. Passen Sie den Code entsprechend an und trainieren Sie das Modell auf dem neuen Datensatz.  

Das Training des Modells benötigt ca. 10 Minuten. Falls das Training zu lange dauert, können Sie auch die Musterlösung laden.

In [ ]:
lr = 0.001 # Lernrate
num_epochs = 40 # Anzahl Epochen 
batch_size = 64 # Batch size

# Gewichtung der Fehler 
w_1 = 10.0 # Lokalisierung
w_2 = 1.0 # Klassifizierung

# Laden der zufälligen Startgewichte des Netzes  
model.load_weights('model_default_weights.h5')

opt = Adam(learning_rate=lr)

model.compile(loss={'output_bb' : 'mean_squared_error', 'output_class' : 'categorical_crossentropy'}, 
              loss_weights=[w_1,w_2],
              optimizer=opt, 
              metrics=['accuracy'])

model_checkpoint_callback = ModelCheckpoint(
                            filepath="my_model_aug.h5", # diese Zeile wurde bereits geändert, um das alte Modell nicht zu überschreiben
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            save_weights_only=False)

history = model.fit(
                    x_train,
                    (bboxes_train, class_train),
                    batch_size=batch_size,
                    epochs=num_epochs,
                    callbacks=[model_checkpoint_callback],
                    validation_split=0.2)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%load Loesung/Loesung_8_1.py

In [ ]:
# Sie können entweder mit dem von Ihnen trainierten Netz weiterarbeiten oder mit der Musterlösung 

# Von Ihnen trainiertes Netz laden 
model.load_weights('my_model_aug.h5')

# Musterlösung laden 
#model.load_weights('solution_model_aug.h5')

### Schritt 9: Evaluation des neuen Modells

#### Aufgabe 9.1: 

Testen Sie die Performance des neuen Modells, welches auf den originalen und den augmentierten Bildern trainiert wurde,
auf den Testdaten. Wie schneidet das neue Modell gegenüber dem alten Modell ab, welches nur auf den originalen Bildern trainiert wurde? 


In [ ]:
# Evaluation der Lokalisierung auf den Testdaten 
threshold = 0.5 # Schwellwert IoU, ab dem Bounding Box als korrekt vorhergesagt betrachtet wird 

box_true = bboxes_test
box_pred = model.predict(x_test)[0]

eval_bbox_prediction(box_true,box_pred,threshold)

# Evaluation der Klassifikation
eval_class_prediction(class_test, model.predict(x_test)[1])

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_9_1.py

#### Aufgabe 9.2:

Testen Sie nun, wie sich die Performance des neuen Modells verhält, wenn man Rauschen auf die Bilder legt. 

a) Experimentieren Sie mit der Menge an Rauschen. Wie schneidet das neue Modell im Vergleich mit dem alten Modell ab?

b) Bei der Augmentierung der Bilder wurde Rauschen in Form von sogenanntem 'salt and pepper noise' (mode='s&p') zu den Bildern hinzugefügt. Es gibt aber auch andere Arten von Rauschen. Wie gut ist die Performance des neuen Modells, wenn man die Art des Rauschens in 'gaussian' oder 'poisson' ändert? Was sagt dies über die Robustheit des Modells aus?    


In [ ]:
amount_of_noise = 0.001 # Menge an Rauschen

def noisy_image(input_imgs):
    noisy_imgs = np.copy(input_imgs)
    for i in range(len(input_imgs)):
        noisy_imgs[i] = random_noise(noisy_imgs[i], mode='s&p', clip=True, amount=amount_of_noise)
        #noisy_imgs[i] = random_noise(noisy_imgs[i], mode='gaussian')
        #noisy_imgs[i] = random_noise(noisy_imgs[i], mode='poisson')    
    return noisy_imgs

x_test_noisy = noisy_image(x_test)

# Plotte Beispielbild mit Rauschen
plt.imshow(x_test_noisy[0], origin='lower')

In [ ]:
eval_bbox_prediction(bboxes_test,model.predict(x_test_noisy)[0],threshold=0.5)
eval_class_prediction(class_test, model.predict(x_test_noisy)[1])

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe a) laden. 

In [ ]:
%run Loesung/Loesung_9_2_a.py

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe b) laden. 

In [ ]:
%run Loesung/Loesung_9_2_b.py

### Anwendung auf dem realen Datensatz - Objekt- und Lageerkennung für Robotergreifen

Im Folgenden sollen die, anhand des simplen synthetischen Datensatzes kennengelernten, Ansätze und Methoden auf einen realen Datensatz übertragen werden. Ziel ist es, eine Objekt- und Lageerkennung für das Greifen unterschiedlicher, chaotisch bereitgestellter Objekte umzusetzen. Bei den Objekten handelt es sich um unterschiedliche Werkzeuge, die vom Roboter von einem Tisch gegriffen und anschließend an definierter Stelle wieder abgelegt werden sollen.  

Eine Einführung in den behandelten Anwendungsfall und die verwendeten Daten wird im Kurselement "Anwendungsfall Robotergreifen" gegeben. Sie können gerne noch einmal zu diesem Kurselement zurückkehren, bevor Sie mit der praktischen Übung fortfahren.

### Schritt 10: Exploration des realen Datensatzes

#### Aufgabe 10.1: 
  
Machen Sie sich zunächst mit dem realen Datensatz vertraut. Wie viele Bilder beinhaltet der Datensatz? In welcher Form werden Bilder und Labels hier gespeichert? Wie viele verschiedene Klassen beinhaltet der Datensatz? Betrachten Sie dazu auch beispielhaft einige der realen Bilder.   

In [ ]:
# Angleichen des realen Datensatzes an die bekannte Datenstruktur 
%run data_transfer.py

In [ ]:
# Plotten einiger Beispielbilder
plot_example_imgs(imgs,bboxes,classes)

# Datenstruktur
print('imgs:', imgs.shape)
print('bboxes:', bboxes.shape)
print('classes:', classes.shape)

# Klassen
print('Klassen:',class_labels)

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_10_1.py

### Schritt 11: Datenvorverarbeitung für den realen Datensatz

#### Aufgabe 11.1: 

Welche Vorverarbeitungsschritte wurden für den synthetischen Datensatz durchgeführt? Führen Sie die gleichen Vorverarbeitungsschritt für den realen Datensatz durch. Ergänzen Sie dazu den gegebenen Code.  

In [ ]:
# Vorverarbeitungsschritt 1
imgs_scaled = 

# Vorverarbeitungsschritt 2
bboxes = 

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%load Loesung/Loesung_11_1.py

### Schritt 12: Sampling für den realen Datensatz

#### Aufgabe 12.1: 

Als Nächstes muss der reale Datensatz in Trainingsdaten und Testdaten aufgeteilt werden. Dabei sollen 80 % der Daten als Trainingsdaten und 20 % der Daten als Testdaten verwendet werden. 

a) Ergänzen Sie den fehlenden Code.

b) Warum ist es hier wichtig, die Daten vor der Aufteilung zu durchmischen? Werfen Sie dazu einen Blick in den Unterordner images im Ordner simple-tools-images-main. 

In [ ]:
# Aufteilung der Daten in Trainings- und Testdaten 

# Durchmischen der Daten vor der Aufteilung  
imgs_scaled = np.array(imgs_scaled)
imgs_scaled, bboxes, classes = shuffle(imgs_scaled, bboxes, classes)

n_train = #Anzahl der Bilder, die zum Trainingsdatensatz gehören sollen

# Trainingsdaten
x_train = # Die ersten n_train Bilder gehören zum Trainingsdatensatz
bboxes_train = 
class_train = 

# Testdaten 
x_test = # Die restlichen Bilder gehören zum Testdatensatz 
bboxes_test =
class_test = 

print("Anzahl Bilder Trainingsdatensatz: ", len(x_train))
print("Anzahl Bilder Testdatensatz: ", len(x_test))

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe a) laden. 

In [ ]:
%load Loesung/Loesung_12_1_a.py

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe b) laden. 

In [ ]:
%run Loesung/Loesung_12_1_b.py

### Schritt 13: Augmentierung des realen Datensatzes

#### Aufgabe 13.1:

Führen Sie den folgenden Code aus, um die Trainingsdaten mit augmentierten Bildern anzureichern und betrachten Sie einige Beispielbilder aus dem neuen Trainingsdatensatz.     

In [ ]:
# Erzeugung augmentierter Bilddaten 
aug_imgs, aug_bboxes, aug_classes = create_augmented_imgs(x_train, bboxes_train, class_train)

# Erstellung eines neuen Trainingsdatensatzes mit den originalen und den augmentierten Bildern  
x_train_aug = np.concatenate((x_train,aug_imgs), axis=0)
bboxes_train_aug = np.concatenate((bboxes_train,aug_bboxes), axis=0)
class_train_aug = np.concatenate((class_train,aug_classes), axis=0)

# Durchmischung der Bilder
x_train_aug, bboxes_train_aug, class_train_aug = shuffle(x_train_aug, bboxes_train_aug, class_train_aug)

# Plotten einiger Beispielbilder
N,h,b,c = aug_imgs.shape
size = np.array([b,h,b,h])
plot_example_imgs(aug_imgs, aug_bboxes*size, aug_classes)

### Schritt 14: Umsetzung Objekterkennung für den realen Datensatz

#### Aufgabe 14.1:

Im Folgenden soll eine Objekterkennung mit einem Convolutional Neural Network (CNN) für den realen Datensatz umgesetzt werden. Dafür soll das in Aufgabe 5.1 definierte Modell wiederverwendet werden. Der Code aus Aufgabe 5.1 ist im Folgenden gegeben. Vergleichen Sie die Struktur des realen Datensatzes und des synthetischen Datensatzes. Kann das Modell aus Aufgabe 5.1 eins zu eins so verwendet werden? 

In [ ]:
def build_model():

    # Eingabe
    inputs = Input(shape=x_train.shape[1:])
    
    # CNN
    x = Convolution2D(filters=32,kernel_size=(3,3), activation="relu")(inputs)
    x = MaxPool2D(pool_size=(3,3))(x)
    x = Convolution2D(filters=16,kernel_size=(3,3), activation="relu")(x)
    x = MaxPool2D(pool_size=(3,3))(x)
    x = Convolution2D(filters=8,kernel_size=(3,3), activation="relu")(x)
    x = MaxPool2D(pool_size=(3,3)) (x)
    x = Flatten() (x)

    # Lokalisierung (Regressionsproblem)
    x_1 = Dense(128, activation="relu")(x)
    x_1 = Dropout(0.3)(x_1)
    x_1 = Dense(64,  activation="relu")(x_1)
    x_1 = Dropout(0.3)(x_1)
    output_bb = Dense(4, activation="sigmoid", name='output_bb')(x_1) # Ausgabe Bounding Box

    # Klassifizierung (Klassifikationsproblem)
    x_2 = Dense(64, activation="relu")(x)
    x_2 = Dropout(0.3)(x_2)
    x_2 = Dense(32, activation="relu")(x_2)
    x_2 = Dropout(0.3)(x_2)
    output_class = Dense(3, activation="softmax", name='output_class')(x_2) # Ausgabe Wahrscheinlichkeit Objektklasse

    model = Model(inputs=inputs, outputs=[output_bb, output_class])
    return model

model = build_model()

# Speichern der zufällig initialisierten Gewichte des Modells 
model.save_weights('real_model_default_weights.h5')

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Teilaufgabe a) laden. 

In [ ]:
%run Loesung/Loesung_14_1.py

#### Aufgabe 14.2: 

Trainieren Sie das Modell auf den augmentierten Trainingsdaten für den realen Anwendungsfall. 

Das Training des Modells benötigt ca. 10 Minuten. Falls das Training zu lange dauert, können Sie auch die Musterlösung laden.

In [ ]:
lr = 0.001 # Lernrate
num_epochs = 40 # Anzahl Epochen
batch_size = 64 # Batch size

# Gewichtung der Fehler 
w_1 = 10.0 # Lokalisierung
w_2 = 1.0 # Klassifizierung

# Laden der zufälligen Startgewichte des Netzes  
model.load_weights('real_model_default_weights.h5')

opt = Adam(learning_rate=lr)

model.compile(loss={'output_bb' : 'mean_squared_error', 'output_class' : 'categorical_crossentropy'}, 
              loss_weights=[w_1,w_2],
              optimizer=opt, 
              metrics=['accuracy'])

model_checkpoint_callback = ModelCheckpoint(
                            filepath="my_real_model_aug.h5",
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            save_weights_only=False)

history = model.fit(
                    x_train_aug,
                    (bboxes_train_aug, class_train_aug),
                    batch_size=batch_size,
                    epochs=num_epochs,
                    callbacks=[model_checkpoint_callback],
                    validation_split=0.2)

In [ ]:
# Sie können entweder mit dem von Ihnen trainierten Netz weiterarbeiten oder mit der Musterlösung 

# Von Ihnen trainiertes Netz laden 
model.load_weights('my_real_model_aug.h5')

# Musterlösung laden 
#model.load_weights('solution_real_model_aug.h5')

### Schritt 15: Evaluation der Objekterkennung für den realen Datensatz

#### Aufgabe 15.1:

Testen Sie die Performance des trainierten Modells auf dem Testdatensatz. Wie gut funktioniert die Lokalisierung und die Klassifikation der Werkzeuge?

In [ ]:
# Evaluation der Lokalisierung auf den Testdaten 
threshold = 0.5 # Schwellwert IoU, ab dem Bounding Box als korrekt vorhergesagt betrachtet wird 

box_true = bboxes_test
box_pred = model.predict(x_test)[0]

eval_bbox_prediction(box_true,box_pred,threshold)

# Evaluation der Klassifikation auf den Testdaten
eval_class_prediction(class_test, model.predict(x_test)[1])

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_15_1.py

#### Aufgabe 15.2:

Insbesondere bei der Lokalisierung gibt es in Bezug auf die Performance des Modells noch Luft nach oben. Überlegen Sie sich, wie die Performance des Modells möglicherweise verbessert werden könnte.  

#### Lösung
Indem Sie die nächste Zeile ausführen, können Sie die Lösung der Aufgabe laden. 

In [ ]:
%run Loesung/Loesung_15_2.py